In [61]:
import os

import time

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pandas as pd


import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from pydub import AudioSegment
import pydub
import moviepy.editor as mp
import wave
import contextlib
#Padding the data
from scipy.io.wavfile import write
from librosa import load
from librosa.util import fix_length

import warnings
warnings.filterwarnings("ignore")

# Loading the Dataset and Feature Extraction

Extract audio from video

In [2]:
import os
x = [[],[],[]]
arr = os.listdir("C:\\Users\\Aditya Kamath\\Downloads\\MP\\New Codes\\Final Dataset\\Video")
print(arr)
for i in range(len(arr)):
    #x.append(arr[i].split(".")[0])
    temparr = os.listdir("C:\\Users\\Aditya Kamath\\Downloads\\MP\\New Codes\\Final Dataset\\Video\\" + arr[i])
    for j in range(len(temparr)):
        x[i].append(temparr[j])

['Confident', 'Nervous', 'Neutral']
['01_01_Confident.mp4', '01_02_Confident.mov', '01_03_Confident.mp4', '01_04_Confident.mp4', '01_05_Confident.mp4', '01_06_Confident.mp4', '01_07_Confident.mp4', '01_08_Confident.mp4', '01_09_Confident.mp4', '01_10_Confident.mp4', '01_11_Confident.mp4', '01_12_Confident.mp4', '01_13_Confident.mp4', '01_14_Confident.mp4', '01_15_Confident.mp4', '01_16_Confident.mp4', '01_17_Confident.mov', '01_18_Confident.mov', '01_19_Confident.mp4', '01_20_Confident.MP4', '01_21_Confident.mp4', '01_22_Confident.MP4', '01_23_Confident.mp4', '01_24_confident.mp4', '01_25_Confident.mp4', '01_26_Confident.mp4', '02_01_Confident.mp4', '02_02_Confident.mov', '02_03_Confident.mp4', '02_04_Confident.mp4', '02_05_Confident.mp4', '02_06_Confident.mp4', '02_07_Confident.mp4', '02_08_Confident.mp4', '02_09_Confident.mp4', '02_10_Confident.mp4', '02_11_Confident.mp4', '02_12_Confident.mp4', '02_13_Confident.mp4', '02_14_Confident.mp4', '02_15_Confident.mp4', '02_16_Confident.mp4

In [8]:
emotions={
  'Confident':'Confident',
  'Neutral':'Neutral',
  'Nervous':'Non Confident',
}
observed_emotions=['Confident', 'Neutral', 'Non Confident']
data_features = []

In [11]:
#- - Extract features (mfcc, chroma, mel) from a sound file
start_time = time.time()
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("Final Dataset\\Padded Audio\\*\\*.wav"):
        file_name=os.path.basename(file)
        x1 = file_name
        x1 = x1.split("_")
        x1 = x1[2].split(".wav")
        emotion=emotions[x1[0]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=False, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)
    
x_train,x_test,y_train,y_test=load_data(test_size=0.25)
print("Shape of training and testing data: ",(x_train.shape[0], x_test.shape[0]))
print(f'Features extracted: {x_train.shape[1]}')

seconds = time.time() - start_time
time_counter=time.strftime("%H:%M:%S",time.gmtime(seconds))
print('Time Taken:', time_counter)

Shape of training and testing data:  (292, 98)
Features extracted: 180
Time Taken: 00:02:30


# Standard SVM

In [62]:
kernel_name = ['linear', 'poly', 'rbf', 'sigmoid']
kernel_train_score = []
kernel_test_score = []
for i in range(len(kernel_name)):
    clf = SVC(kernel=kernel_name[i])
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))


data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_train_score))]}

df = pd.DataFrame(data, index =['linear', 'poly', 'rbf', 'sigmoid'])
df

,Train Score,Test Score
linear,81.16%,61.22%
poly,41.1%,41.84%
rbf,39.73%,42.86%
sigmoid,34.59%,29.59%


In [63]:
kernel_name = 'linear'
degree_name = [1,2,3,4,5]
kernel_train_score = []
kernel_test_score = []
for i in range(len(degree_name)):
    clf = SVC(kernel=kernel_name,degree=degree_name[i])
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["degree =1","degree =2","degree =3","degree =4","degree =5"])
df

,Train Score,Test Score
degree =1,81.16%,61.22%
degree =2,81.16%,61.22%
degree =3,81.16%,61.22%
degree =4,81.16%,61.22%
degree =5,81.16%,61.22%


In [100]:
kernel_name = 'poly'
degree_name = 3
gamma_name = ["scale","auto"]
kernel_train_score = []
kernel_test_score = []
for i in range(len(gamma_name)):
    clf = SVC(kernel=kernel_name,degree=degree_name,gamma=gamma_name[i])
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["scale","auto"])
print("kernel is poly")
df

kernel is poly


,Train Score,Test Score
scale,41.1%,41.84%
auto,84.25%,48.98%


In [101]:
kernel_name = 'rbf'
degree_name = 3
gamma_name = ["scale","auto"]
kernel_train_score = []
kernel_test_score = []
for i in range(len(gamma_name)):
    clf = SVC(kernel=kernel_name,degree=degree_name,gamma=gamma_name[i])
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["scale","auto"])
print("kernel is rbf")
df

kernel is rbf


,Train Score,Test Score
scale,39.73%,42.86%
auto,91.1%,46.94%


In [102]:
kernel_name = 'sigmoid'
degree_name = 3
gamma_name = ["scale","auto"]
kernel_train_score = []
kernel_test_score = []
for i in range(len(gamma_name)):
    clf = SVC(kernel=kernel_name,degree=degree_name,gamma=gamma_name[i])
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["scale","auto"])
print("kernel is sigmoid")
df

kernel is sigmoid


,Train Score,Test Score
scale,34.59%,29.59%
auto,34.59%,29.59%


In [104]:
kernel_name = 'linear'
degree_name = 3
gamma_name = ["scale","auto"]
kernel_train_score = []
kernel_test_score = []
for i in range(len(gamma_name)):
    clf = SVC(kernel=kernel_name,degree=degree_name,gamma=gamma_name[i])
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["scale","auto"])
print("kernel is linear")
df

kernel is linear


,Train Score,Test Score
scale,81.16%,61.22%
auto,81.16%,61.22%


In [105]:
kernel_name = 'linear'
degree_name = 3
decision_function_name = ['ovo','ovr']
kernel_train_score = []
kernel_test_score = []
for i in range(len(decision_function_name)):
    clf = SVC(kernel=kernel_name,degree=degree_name,gamma="auto",decision_function_shape=decision_function_name[i])
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["ovo","ovr"])
df

,Train Score,Test Score
ovo,81.16%,61.22%
ovr,81.16%,61.22%


# Piplined svm

In [65]:
kernel_name = ['linear', 'poly', 'rbf', 'sigmoid']
kernel_train_score = []
kernel_test_score = []
for i in range(len(kernel_name)):
    clf = make_pipeline(StandardScaler(), SVC(kernel=kernel_name[i]))
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))


data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_train_score))]}

df = pd.DataFrame(data, index =['linear', 'poly', 'rbf', 'sigmoid'])
df

,Train Score,Test Score
linear,92.81%,59.18%
poly,53.77%,44.9%
rbf,74.66%,63.27%
sigmoid,57.88%,50.0%


In [66]:
kernel_name = 'linear'
degree_name = [1,2,3,4,5]
kernel_train_score = []
kernel_test_score = []
for i in range(len(degree_name)):
    clf = make_pipeline(StandardScaler(),SVC(kernel=kernel_name,degree=degree_name[i]))
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["degree =1","degree =2","degree =3","degree =4","degree =5"])
df

,Train Score,Test Score
degree =1,92.81%,59.18%
degree =2,92.81%,59.18%
degree =3,92.81%,59.18%
degree =4,92.81%,59.18%
degree =5,92.81%,59.18%


In [106]:
kernel_name = 'linear'
degree_name = 3
gamma_name = ["scale","auto"]
kernel_train_score = []
kernel_test_score = []
for i in range(len(gamma_name)):
    clf = make_pipeline(StandardScaler(),SVC(kernel=kernel_name,degree=degree_name,gamma=gamma_name[i]))
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["scale","auto"])
print("Kernal is Liner")
df

Kernal is Liner


,Train Score,Test Score
scale,92.81%,59.18%
auto,92.81%,59.18%


In [108]:
kernel_name = 'poly'
degree_name = 3
gamma_name = ["scale","auto"]
kernel_train_score = []
kernel_test_score = []
for i in range(len(gamma_name)):
    clf = make_pipeline(StandardScaler(),SVC(kernel=kernel_name,degree=degree_name,gamma=gamma_name[i]))
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["scale","auto"])
print("Kernal is poly")
df

Kernal is poly


,Train Score,Test Score
scale,53.77%,44.9%
auto,53.77%,44.9%


In [109]:
kernel_name = 'rbf'
degree_name = 3
gamma_name = ["scale","auto"]
kernel_train_score = []
kernel_test_score = []
for i in range(len(gamma_name)):
    clf = make_pipeline(StandardScaler(),SVC(kernel=kernel_name,degree=degree_name,gamma=gamma_name[i]))
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["scale","auto"])
print("Kernal is rbf")
df

Kernal is rbf


,Train Score,Test Score
scale,74.66%,63.27%
auto,74.66%,63.27%


In [110]:
kernel_name = 'sigmoid'
degree_name = 3
gamma_name = ["scale","auto"]
kernel_train_score = []
kernel_test_score = []
for i in range(len(gamma_name)):
    clf = make_pipeline(StandardScaler(),SVC(kernel=kernel_name,degree=degree_name,gamma=gamma_name[i]))
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["scale","auto"])
print("Kernal is sigmoid")
df

Kernal is sigmoid


,Train Score,Test Score
scale,57.88%,50.0%
auto,57.88%,50.0%


In [111]:
kernel_name = 'linear'
degree_name = 3
decision_function_name = ['ovo','ovr']
kernel_train_score = []
kernel_test_score = []
for i in range(len(decision_function_name)):
    clf = make_pipeline(StandardScaler(),SVC(kernel=kernel_name,degree=degree_name,gamma="auto",decision_function_shape=decision_function_name[i]))
    clf.fit(x_train,y_train)
    kernel_train_score.append(clf.score(x_train, y_train))
    y_pred=clf.predict(x_test)
    kernel_test_score.append(accuracy_score(y_true=y_test, y_pred=y_pred))

import pandas as pd
data = {'Train Score':[str(round(kernel_train_score[i]*100,2))+"%" for i in range(len(kernel_train_score))],
        'Test Score':[str(round(kernel_test_score[i]*100,2))+"%" for i in range(len(kernel_test_score))]}

df = pd.DataFrame(data,index =["ovo","ovr"])
df

,Train Score,Test Score
ovo,92.81%,59.18%
ovr,92.81%,59.18%
